# Challenge 2 - NLP

## Libraries

In [1]:
#Default
import os
import re
import warnings
import numpy as np
import pandas as pd

#Visualization
import seaborn as sns
import matplotlib.pyplot as plt


#NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS

import gensim

#Configuration
sns.set_style("darkgrid")
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Utils Functions

In [2]:
def createDataset(files):
    """
    Creating data from a file .txt
    Params:
    -------
    files: list of the file path that we want to create a dataset
    Return:
    -------
    DataFrame of the dataset we create
    """
    all_data = []
    for idx, file in enumerate(files):
        reader = open(file, mode='r', encoding='utf-8')
        reader = reader.read()
        data = {
            'idx': idx,
            'content': reader
        }
        all_data.append(data)
    
    df = pd.DataFrame(all_data)
    return df

## Load Dataset

In [3]:
data_dir = os.path.join('data')

In [4]:
#Get files from article
files = []
for file_name in os.listdir(data_dir):
    if file_name.endswith('.txt'):
        files.append(os.path.join(data_dir, file_name))

## Create Dataset

In [5]:
df_ch2 = createDataset(files)

In [6]:
df_ch2.head()

,idx,content
0,0,This is when the arthritis starts. A cheese t...
1,1,"Just like the other animals on this list, the..."
2,2,No better way to spend your days off than rese...
3,3,"In my garden, I grow a lot of medicinal plants..."
4,4,I was hiking through the canyon and found this...


## Data Analyse

In [7]:
df_ch2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   idx      145 non-null    int64 
 1   content  145 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [8]:
# Check missing values
df_ch2.isnull().sum()

idx        0
content    0
dtype: int64

## Data Preprocessing